Today i am working on an amazon dataset from strata under presumably hard difficulty.

Here is the prompt:

You have a table of in-app purchases by user. Users that make their first in-app purchase are placed in a marketing campaign where they see call-to-actions for more in-app purchases. Find the number of users that made additional in-app purchases due to the success of the marketing campaign.

The marketing campaign doesn't start until one day after the initial in-app purchase so users that only made one or multiple purchases on the first day do not count, nor do we count users that over time purchase only the products they purchased on the first day.

user\_id:intcreated\_at:datetimeproduct\_id:intquantity:intprice:int

Solution

Summary of approach is to group users where the product\_id after the min(created\_at) or date of app purchase is NOT EQUAL to the product\_id

This is based on the conditions given:

date of purchase (min(created\_at)) is not considered in the in-app purchases

and same product id after the required date is not accepted.

In [ ]:
-- see the min and max time per user_id ordered in asc
select user_id, min(created_at), max(created_at)
from marketing_campaign
group by user_id
order by user_id asc

In [ ]:
with minimum_time as(
    select user_id, min(created_at) as mc_start
    from marketing_campaign
    group by user_id
    order by user_id
),

data as (
select mc.user_id, sum(quantity) as sum_quantity, sum(price) as sum_price
from marketing_campaign as mc
join minimum_time as mt
on mc.user_id = mt.user_id

where mc.created_at > mt.mc_start


group by mc.user_id
order by mc.user_id
)

select sum_quantity,
COUNT(*)
from data
where sum_quantity > 0
group by sum_quantity

In [ ]:
/*select *,
rank() over(partition by user_id 
                         order by created_at) as rn,
    concat((user_id),'_', (product_id)) as user_product
from marketing_campaign
where user_id = 25



with minimum_time as(
    select user_id, min(created_at) as mc_start
    from marketing_campaign
    group by user_id
    order by user_id
),

data as (
select mc.user_id, sum(quantity) as sum_quantity, sum(price) as sum_price
from marketing_campaign as mc
join minimum_time as mt
on mc.user_id = mt.user_id

where mc.created_at > mt.mc_start
or mc.created_at != mc.created_at
or mc.product_id != mc.product_id

group by mc.user_id
order by mc.user_id
)

select *
from data;
/*
select sum_quantity,
COUNT(*)
from data
where sum_quantity > 0
group by sum_quantity


--practice on using the having function